In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span" , attrs = {'class':"a-offscreen"}).text.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span" , attrs = {'class':"a-offscreen"}).text.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon-alt'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [3]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36 Edg/124.0.0.0', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=playstation+5&crid=2TRY1AMWDZJFG&sprefix=plays%2Caps%2C334&ref=nb_sb_ss_ts-doa-p_1_5"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.in" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [ ]:
amazon_df

,title,price,rating,reviews,availability
0,Sony PS5 Console,"₹44,990.00",4.6 out of 5 stars,"1,448 ratings",In stock
1,Sony PlayStation®5 Console (slim),"₹49,990.00",3.8 out of 5 stars,11 ratings,In stock
2,Sony God Of War Ragnarok | Standard Edition | ...,"₹2,999.00",4.7 out of 5 stars,"4,258 ratings",In stock
3,Sony PS5 PlayStation Console+God Of War Ragnar...,"₹47,989.00",4.6 out of 5 stars,67 ratings,In stock
4,2K Games Wwe 2K24 | Standard Edition | Playsta...,"₹4,450.00",4.4 out of 5 stars,8 ratings,In stock
5,Far Cry 6 - PlayStation 5 Standard Edition,"₹1,812.00",3.9 out of 5 stars,35 ratings,In stock
6,Sony Horizon Forbidden West | Standard Edition...,Page 1 of 1,4.6 out of 5 stars,446 ratings,Not Available
7,Sniper Elite 5 - Playstation 5,"₹3,022.00",4.6 out of 5 stars,544 ratings,Only 1 left in stock.
8,Sony PS5 Console - EA SPORTS FC24 Bundle,"₹56,999.00",4.6 out of 5 stars,127 ratings,In stock
9,Sony DualSense Wireless Controller White (Play...,"₹5,749.00",4.5 out of 5 stars,"3,866 ratings",In stock


In [ ]:
amazon_df.to_csv("C:\\Users\\swain\\OneDrive\\Desktop\\Outfut_file\\final_amazon.csv")